In [2]:
# ! pip install selenium

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 904 kB 2.7 MB/s eta 0:00:01
You should consider upgrading via the '/usr/local/anaconda3/envs/test/bin/python -m pip install --upgrade pip' command.


In [1]:
from bs4 import BeautifulSoup
import requests
import urllib3
from pymongo import MongoClient
import re
from multiprocessing import Pool
import traceback
from selenium import webdriver
import time
import pandas as pd
import os

In [2]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [1]:
! nvidia-smi

Sun Mar  6 12:57:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 28%   36C    P8    20W / 260W |     22MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 26%   

# Soup

In [4]:
def handle_error(e):
    #處理 child process 的錯誤，不然 code 寫錯時，不會回報任何錯誤
    traceback.print_exception(type(e), e, e.__traceback__)

In [5]:
def patent_soup(patent_num):
    urllib3.disable_warnings() #禁用各種警告
    response = requests.get('https://patents.google.com/patent/' + str(patent_num),verify = False)
    response.encoding='utf-8'
    if response.status_code != 200:
        pass
    else:
        soup =  BeautifulSoup(response.text, 'html.parser') 
        try:
            appDate = soup.find('time', itemprop="filingDate").string.replace('-', '')
            
            # Classifications
            try:
                classifications = soup.find_all('span', itemprop="Code")
                classification =[]
                for i in range(0,len(classifications)):
                    try:
                        if len(classifications[i].string)>len(classifications[i+1].string):
                            classification.append(classifications[i].string)
                        elif i == len(classifications)-2:
                            classification.append(classifications[i+1].string) 
                    except:
                        pass
                if classification[0] == '':
                    classification = None
            except:
                classification = None
        except:
            return None
        
    return appDate, classification

In [6]:
# 取得html
def soup(url):
    urllib3.disable_warnings() #禁用各種警告
    response = requests.get(url,verify = False)
    response.encoding='utf-8'
    if response.status_code != 200:
        pass
    else:
        soup =  BeautifulSoup(response.text, 'html.parser') 
        try:
            patent_num = soup.find('dd',itemprop='publicationNumber').string
            appDate = soup.find('time', itemprop="filingDate").string.replace('-', '')
            pubDate = soup.find('time', itemprop="publicationDate").string.replace('-', '')
            abstract = soup.find('div',{"class":"abstract"}).string
            title = re.sub('\s','',soup.find('span',itemprop="title").string)
            
            #current assignee
            try:
                current = soup.find_all('dd' ,itemprop="assigneeCurrent")
                current_assignee = []
                for i in current:
                    i = re.sub('\n','',i.string)
                    i = ' '.join(i.split())
                    current_assignee.append(i)
            except:
                current_assignee = None
            
            #original assignee
            try:
                original = soup.find_all('dd' ,itemprop="assigneeOriginal")
                original_assignee = []
                for i in original:
                    original_assignee.append(i.string)
            except:
                original_assignee = None
            
            #graDate
            Date = soup.find_all('tr',itemprop="legalEvents")
            graDate = 'N/A'
            for i in Date:
#                 if re.search('.*Grant.*',i.text) != None or re.search('.*grant.*',i.text) != None:
#                     graDate =re.sub('\s','',i.text)[:10].replace('-', '')
#                 else:
#                     graDate = graDate
                if i.find('td', itemprop='title').text == 'Information on status: patent grant':
                    graDate =re.sub('\s','',i.text)[:10].replace('-', '')
            
            # inventorName
            try:
                inventorNames = soup.find_all('dd',itemprop="inventor")
                inventorName = []
                for i in inventorNames:
                    inventorName.append(i.string)
            except:
                inventorName = None

            # description
#             try:
#                 description = soup.find_all('p')
#                 description_list = []
#                 for i in description:
#                     description_list.append(re.sub('\s','',i.text))
#                 description = ("".join(description_list)) 
#             except:
#                 description = None
            if graDate != 'N/A':
                try:
                    descriptions = soup.find_all('div', class_="description-paragraph") #CN
                    description = ''
                    for i in descriptions:
                        description += i.text
                except:
                    description = None
            else:
                try:
                    descriptions = soup.find_all('div', class_="description-line")
                    description = ''
                    if len(descriptions) != 0:
                        for i in descriptions:
                            description += i.text
                    else:
                        descriptions = soup.find_all('div', class_="description-paragraph") #CN
                        for i in descriptions:
                            description += i.text
                except:
                    description = None
                                
            # detailed_description
#             try:
#                 detailed_description_list = soup.find_all('p')
#                 detailed_description_start = 0
#                 detailed_description = ''
                            
#                 for num,i in enumerate(detailed_description_list):
#                     if re.sub('\s','',i.text) == '具体实施方式':
#                         detailed_description_start = num
#                 if detailed_description_start > 0:
#                     for i in range(detailed_description_start,len(detailed_description_list)):
#                         detailed_description += re.sub('\s','',detailed_description_list[i].text)
#                     detailed_description = "".join(detailed_description)
#                 else:
#                     detailed_description = None
#             except:
#                 detailed_description = None
                
            # claim
            try:
                claims = soup.find_all('div',{'class':'claim-text'})
                claim = []
                j = 0
                for i in claims:
                    if re.sub('\s','',i.text) != '':
                        claim.append(re.sub('\s','',i.text))
            #分類
                for i in range(1,len(claim)-1):
                    try:
                        int(claim[i][0])
                        j = i
                    except:
                        a = str(claim[j])+str(claim[i])
                        claim[j] = a
                        claim[i] = '' 
                for i in claim:
                    if '' in claim:
                        claim.remove('')
            except:
                claim = None

            # patentCited    
            try:
                patentCiteds = soup.find_all('tr', itemprop="backwardReferences")
                patentCited_list = []
                patentCited_assignee_list = []
                patentCited = []
                for i in patentCiteds:
                    patentCited_assignee_list.append(i.find('span', itemprop="assigneeOriginal").text)
                    e = re.sub('\s','',i.text).find("(")
                    patentCited_list.append(re.sub('\s','',i.text)[:e])
                if len(patentCited_list) != 0 :
                    for j in patentCited_list:
                        pclist = {'pto':None,'patentNumber':None,'kindcode':None, 'appDate':None, 'assignee':None, 'classification':None}
                        pclist['pto'] = j[:2]
                        try:
                            int(j[-2])
                            try:
                                int(j[-1])
                                pclist['patentNumber'] = j[2:]
                                pclist['kindcode'] = None
                            except:
                                pclist['patentNumber'] = j[2:-2]
                                pclist['kindcode'] = j[-1]
                        except:
                            pclist['patentNumber'] = j[2:-2]
                            pclist['kindcode'] = j[-2:]
                        try:                   
                            cited_appDate, cited_classification = patent_soup(j)
                            pclist['appDate'] = cited_appDate
                            pclist['classification'] = cited_classification
                        except:
                            pclist['appDate'] = None
                            pclist['classification'] = None
                        patentCited.append(pclist)
                    for i in range(len(patentCited)):
                        try:
                            patentCited[i]['assignee'] = patentCited_assignee_list[i]
                        except:
                            patentCited[i]['assignee'] = None
                else:
                    patentCited = None
                    
            except:
                patentCited = None
                
            # Family To Family Citing      
            try:
                family_to_family_citings = soup.find_all('tr', itemprop="backwardReferencesFamily")
                family_to_family_citing_list = []
                family_to_family_citing_assignee_list = []
                family_to_family_citing = []
                for i in family_to_family_citings:
                    family_to_family_citing_assignee_list.append(i.find('span', itemprop="assigneeOriginal").text)
                    e = re.sub('\s','',i.text).find("(")
                    family_to_family_citing_list.append(re.sub('\s','',i.text)[:e])
                    
                if len(family_to_family_citing_list) != 0 :
                    for j in family_to_family_citing_list:
                        clist = {'pto':None,'patentNumber':None,'kindcode':None, 'appDate':None, 'assignee':None, 'classification':None}
                        clist['pto'] = j[:2]
                        try:
                            int(j[-2])
                            try:
                                int(j[-1])
                                clist['patentNumber'] = j[2:]
                                clist['kindcode'] = None
                            except:
                                clist['patentNumber'] = j[2:-2]
                                clist['kindcode'] = j[-1]
                        except:
                            clist['patentNumber'] = j[2:-2]
                            clist['kindcode'] = j[-2:]
                        try:                   
                            citing_appDate, citing_classification = patent_soup(j)
                            clist['appDate'] = citing_appDate
                            clist['classification'] = citing_classification
                        except:
                            clist['appDate'] = None
                            clist['classification'] = None
                        family_to_family_citing.append(clist) 
                    for i in range(len(family_to_family_citing)):
                        try:
                            family_to_family_citing[i]['assignee'] = family_to_family_citing_assignee_list[i]
                        except:
                            family_to_family_citing[i]['assignee'] = None
                else:
                    family_to_family_citing = None
            except:
                family_to_family_citing = None
                
            # ＣountryStatus
            try:
                status =  soup.find_all('tr', itemprop="countryStatus")
                countrystatus = []
                for i in status:
                    countrycode = i.find('span', itemprop="countryCode")
                    try:
                        countrystatus.index(countrycode.string)
                    except:
                        countrystatus.append(countrycode.string)
            except:
                countrystatus = None
            
            # Cited       
            try:
                citeds_1 = soup.find_all('tr' ,itemprop="forwardReferencesOrig" )
                #citeds_2 = soup.find_all('tr', itemprop="forwardReferencesFamily")
                cited_list = []
                cited_assignee_list = []
                cited = []
                for i in citeds_1:
                    cited_assignee_list.append(i.find('span', itemprop="assigneeOriginal").text)
                    e = re.sub('\s','',i.text).find("(")
                    cited_list.append(re.sub('\s','',i.text)[:e])
#                 for i in citeds_2:
#                     e = re.sub('\s','',i.text).find("(")
#                     cited_list.append(re.sub('\s','',i.text)[:e])
                    
                if len(cited_list) != 0 :
                    for j in cited_list:
                        clist = {'pto':None,'patentNumber':None,'kindcode':None, 'appDate':None, 'assignee':None, 'classification':None}
                        clist['pto'] = j[:2]
                        try:
                            int(j[-2])
                            try:
                                int(j[-1])
                                clist['patentNumber'] = j[2:]
                                clist['kindcode'] = None
                            except:
                                clist['patentNumber'] = j[2:-2]
                                clist['kindcode'] = j[-1]
                        except:
                            clist['patentNumber'] = j[2:-2]
                            clist['kindcode'] = j[-2:]
                        try:                   
                            cited_appDate, cited_classification = patent_soup(j)
                            clist['appDate'] = cited_appDate
                            clist['classification'] = cited_classification
                        except:
                            clist['appDate'] = None
                            clist['classification'] = None
                        cited.append(clist)
                    for i in range(len(cited)):
                        try:
                            cited[i]['assignee'] = cited_assignee_list[i]
                        except:
                            cited[i]['assignee'] = None
                
                else:
                    cited = None
            except:
                cited = None
                
            # Family To Family Cited      
            try:
                family_to_family_citeds = soup.find_all('tr', itemprop="forwardReferencesFamily")
                family_to_family_cited_list = []
                family_to_family_cited_assignee_list = []
                family_to_family_cited = []
                for i in family_to_family_citeds:
                    family_to_family_cited_assignee_list.append(i.find('span', itemprop="assigneeOriginal").text)
                    e = re.sub('\s','',i.text).find("(")
                    family_to_family_cited_list.append(re.sub('\s','',i.text)[:e])
                    
                if len(family_to_family_cited_list) != 0 :
                    for j in family_to_family_cited_list:
                        clist = {'pto':None,'patentNumber':None,'kindcode':None, 'appDate':None, 'assignee':None, 'classification':None}
                        clist['pto'] = j[:2]
                        try:
                            int(j[-2])
                            try:
                                int(j[-1])
                                clist['patentNumber'] = j[2:]
                                clist['kindcode'] = None
                            except:
                                clist['patentNumber'] = j[2:-2]
                                clist['kindcode'] = j[-1]
                        except:
                            clist['patentNumber'] = j[2:-2]
                            clist['kindcode'] = j[-2:]
                        try:                   
                            cited_appDate, cited_classification = patent_soup(j)
                            clist['appDate'] = cited_appDate
                            clist['classification'] = cited_classification
                        except:
                            clist['appDate'] = None
                            clist['classification'] = None
                        family_to_family_cited.append(clist) 
                    for i in range(len(family_to_family_cited)):
                        try:
                            family_to_family_cited[i]['assignee'] = family_to_family_cited_assignee_list[i]
                        except:
                            family_to_family_cited[i]['assignee'] = None
                else:
                    family_to_family_cited = None
            except:
                family_to_family_cited = None

            # Classifications
            try:
                classifications = soup.find_all('span', itemprop="Code")
                classification =[]
                for i in range(0,len(classifications)):
                    try:
                        if len(classifications[i].string)>len(classifications[i+1].string):
                            classification.append(classifications[i].string)
                        elif i == len(classifications)-2:
                            classification.append(classifications[i+1].string) 
                    except:
                        pass
                if classification[0] == '':
                    classification = None
            except:
                classification = None
                
            # Similar Documents
            try:
                similar_docs = soup.find_all('tr', itemprop="similarDocuments")
                similarDocuments = []
                similar_doc =[]
                doc_title = []
                doc_publication = []
                doc_time = []

                for doc in similar_docs:
                    publication = doc.find_all('span')
                    time = doc.find_all('time')
                    titles = doc.find_all('td', itemprop="title")

                    for j1 in publication:
                        if len(j1.text.strip())!=2:
                            doc_publication.append(j1.text.strip())
                    for j2 in time:
                        doc_time.append(j2.text.strip())
                    for j3 in titles:
                        doc_title.append(j3.text.strip())
                similarDocuments = []
                for i in range(len(doc_publication)):
                    doc={
                        'publication' : doc_publication[i],
                        'time' : doc_time[i],
                        'title' : doc_title[i]
                        }
                    similarDocuments.append(doc)
            except:
                similarDocuments = None
                

            data = {
            'patentNumber':patent_num, 
            'inventorName':inventorName, 
            'current_assignee':current_assignee, 
            'original_assignee':original_assignee,
            'appDate':appDate, # application Date
            'pubDate':pubDate , # public Date
            'graDate':graDate, #
            'classification':classification, # 類別
            'brief':abstract, # 摘要
            'title':title, # 標題
            'description':description, 
#             'detailed_description':detailed_description,
            'claim':claim,
            'countrystatus':countrystatus,
            'patentCitation':patentCited,
            'family_to_family_citing':family_to_family_citing,
            'cited':cited,
            'family_to_family_cited':family_to_family_cited,
            'similarDocuments':similarDocuments
            }
            return data
        except:
            #print("except==",url)
            #error_url.append(url)
            #pass
            
            return None

In [7]:
#搜尋google patent
def search_google_patent(keyword='', assignee=[], s_date='', e_date=''):
    assignees = ''
    for i in assignee:
        for j in i.split():
            assignees += str(j)
            assignees += '+'
        assignees += ','
    url = 'https://patents.google.com/?q=' + keyword + '&assignee=' + assignees + '&country=US&before=filing:' + e_date + '&after=filing:' + s_date + '&language=ENGLISH&type=PATENT'
    
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--disable-infobars')
    options.add_argument('--no-sandbox')

    driver = webdriver.Chrome(chrome_options=options, executable_path='./chromedriver')
    driver.get(url)
    time.sleep(2)
    html = driver.page_source
    driver.close()
    soup = BeautifulSoup(html, "html.parser")
    
    csv_url = 'https://patents.google.com/' + soup.find('a', class_='style-scope search-results')['href']
    urllib.request.urlretrieve(csv_url, "US_patent.csv")

# MongoDB

In [ ]:
from pymongo import MongoClient
from sshtunnel import SSHTunnelForwarder
import pymongo
from tqdm import tqdm

def insert_data(year, data):
    MONGO_HOST = "140.117.69.70"
    MONGO_USER = "eva"
    MONGO_PASS = "eva_30241"
    MONGO_DB = "eva"
    MONGO_COLLECTION = "20" + year + "_Patent_Innovation"
    
    server = SSHTunnelForwarder(
        MONGO_HOST,
        ssh_username=MONGO_USER,
        ssh_password=MONGO_PASS,
        remote_bind_address=('140.117.69.70', 30241)
    )
    
    server.start()

    conn = pymongo.MongoClient('127.0.0.1',server.local_bind_port)
    conn.admin.authenticate('eva', 'eva_30241')
    conn_db = conn[MONGO_DB]
    conn_db_collection = conn_db[MONGO_COLLECTION]
    
    conn_db_collection.insert_one(data)

    server.stop()
    
def SearchYear(year):
    MONGO_HOST = "140.117.69.70"
    MONGO_USER = "eva"
    MONGO_PASS = "eva_30241"
    MONGO_DB = "eva"
    MONGO_COLLECTION = "application_20" + year
    
    server = SSHTunnelForwarder(
        MONGO_HOST,
        ssh_username=MONGO_USER,
        ssh_password=MONGO_PASS,
        remote_bind_address=('140.117.69.70', 30241)
    )
    
    server.start()

    conn = pymongo.MongoClient('127.0.0.1',server.local_bind_port)
    conn.admin.authenticate('eva', 'eva_30241')
    conn_db = conn[MONGO_DB]
    conn_db_collection = conn_db[MONGO_COLLECTION]
    
    #建立索引
    #conn_db_collection.create_index("assignees.orgname", name='orgname', background=True)
    #刪除索引
    #data = conn_db_collection.find_one({"assignees.orgname":'Apple Inc'})
    #data = conn_db_collection.find({"assignees.orgname":'Apple Inc'}).explain()["executionStats"]
    data = list(conn_db_collection.find({}, {'_id':0, 'url':1}))
    #information = conn_db_collection.index_information()
    #information = conn_db_collection.index_information()

    server.stop()
    
    return data

def SearchPatentNumber(year, num):
    MONGO_HOST = "140.117.69.70"
    MONGO_USER = "eva"
    MONGO_PASS = "eva_30241"
    MONGO_DB = "evq"
    MONGO_COLLECTION = "20" + year + "_Patent_Innovation"
    
    server = SSHTunnelForwarder(
        MONGO_HOST,
        ssh_username=MONGO_USER,
        ssh_password=MONGO_PASS,
        remote_bind_address=('140.117.69.70', 30241)
    )
    
    server.start()

    conn = pymongo.MongoClient('127.0.0.1',server.local_bind_port)
    conn.admin.authenticate('eva', 'eva_30241')
    conn_db = conn[MONGO_DB]
    conn_db_collection = conn_db[MONGO_COLLECTION]
    
    conn_db_collection.create_index("patentNumber", background=True)
    data = conn_db_collection.find_one({"patentNumber":num},{'_id':0, 'patentNumber':1})

    server.stop()
    
    return data

In [ ]:
#搜尋patent csv
keyword = ''
assignee = ['Apple Inc.','Google LLC.'] #ex.['Apple Inc.','Google LLC.']
s_date = '20150101'
e_date = '20201231'
search_google_patent(keyword, assignee, s_date, e_date)

In [ ]:
#爬蟲並新增資料
data = pd.read_csv('US.csv', encoding= 'unicode_escape')
urls = list(data.iloc[:, 0])
del urls[0]

for i in tqdm(range(0,10)):
    data = soup(urls[i])
    insert_data(data)

In [ ]:
#資料庫查patent_url抓資料
n_num = 1
for i in range(10, 21):
    patent = SearchYear(str(i))
    print(str(n_num) + "/" + '11')
    for j in tqdm(patent):
        patent_num = j['url'].replace('https://patents.google.com/patent/', '')
        Is_patent_num = SearchPatentNumber(str(i), str(patent_num))
        if Is_patent_num:
            n_num += 1
            continue
        else:
            data = soup(j['url'])
            insert_data(str(i), data)
    n_num += 1

# Test 抓不同國家資料

In [8]:
from pymongo import MongoClient
# from sshtunnel import SSHTunnelForwarder
import pymongo
from tqdm import tqdm

In [11]:
urllib3.disable_warnings() #禁用各種警告
response = requests.get("https://patents.google.com/patent/CN103890730B/zh",verify = False)
response.encoding='utf-8'
if response.status_code != 200:
    pass
else:
    soup =  BeautifulSoup(response.text, 'html.parser') 

In [12]:
soup

<!DOCTYPE doctype html>

<html lang="en">
<head>
<title>CN103890730B - 用于传感器驱动的车辆遥测应用和服务的开发和部署的计算平台 
        - Google Patents</title>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta charset="utf-8"/>
<meta content="origin-when-crossorigin" name="referrer"/>
<link href="https://patents.google.com/patent/CN103890730B/zh" rel="canonical"/>
<meta content="
     本文公开了用于车辆遥测应用的智能开发、部署和管理的计算平台。此外，本发明提供了一种允许在基于云的平台上提供智能交通服务的方法和系统，该基于云的平台助于被配置为允许交通测量、交通整形、车辆监控和其他有关车辆的服务的车辆遥测应用的创建和部署。 
     
   " name="description"/>
<meta content="patent" name="DC.type"/>
<meta content="用于传感器驱动的车辆遥测应用和服务的开发和部署的计算平台 
       " name="DC.title"/>
<meta content="2012-09-18" name="DC.date" scheme="dateSubmitted"/>
<meta content="
     本文公开了用于车辆遥测应用的智能开发、部署和管理的计算平台。此外，本发明提供了一种允许在基于云的平台上提供智能交通服务的方法和系统，该基于云的平台助于被配置为允许交通测量、交通整形、车辆监控和其他有关车辆的服务的车辆遥测应用的创建和部署。 
     
   " name="DC.description"/>
<meta content="CN:201280046934.8A" name="citation_patent_application_number"/>
<meta content="https://p

In [15]:
descriptions = soup.find_all('div', class_="description-line")

In [17]:
len(descriptions)

0

## DE

In [24]:
# MongoDB 連線
def insert_data(data):
    mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    conn_db_collection = db.patent_de
    
    conn_db_collection.insert_one(data) #insert data

In [45]:
# soup(data["result link"][0])

{'patentNumber': 'DE112012005419B4',
 'inventorName': ['Chi Liu',
  'Qi Yu',
  'Zhentan Feng',
  'Bo Yang',
  'Zhanwei Sun'],
 'current_assignee': ['GlobalFoundries Inc', 'GlobalFoundries US Inc'],
 'original_assignee': ['GlobalFoundries Inc'],
 'appDate': '20121031',
 'pubDate': '20190718',
 'graDate': 'N/A',
 'classification': ['G06F1/3234',
  'G06F1/329',
  'H04W52/0216',
  'H04L43/08',
  'H04W52/0219',
  'Y02D30/70'],
 'brief': None,
 'title': 'SystemundVerfahrenfüreineeffizientedienstinstanzenorientierteEnergieverwaltungimInternetderDinge',
 'description': 'Technisches GebietDie vorliegende Erfindung betrifft ein Gebiet des Internet der Dinge (Internet of Things, IoT). Insbesondere betrifft die vorliegende Erfindung ein System und ein Verfahren für eine effiziente dienstinstanzenorientierte Energieverwaltung im IoT.Beschreibung der verwendeten TechnikDas loT ist ein wichtiger Bestandteil der Informationstechnik der neuen Generation und sein vollständiger Name lautet „das Internet 

In [ ]:
#爬蟲並新增資料
data_DE = pd.read_csv('../years/DE/DE_all.csv', encoding= 'unicode_escape')
# urls = list(data.iloc[:, 0])
# del urls[0]

for i in tqdm(range(0,len(data_DE))):
    data_s = soup(data_DE["result link"][i])
    insert_data(data_s)

  1%|          | 18/1833 [09:33<16:24:07, 32.53s/it]

## CN

In [9]:
data_cn = pd.read_csv('../years/CN/CN_all.csv')

In [10]:
data_cn.head(2)

,id,title,assignee,inventor/author,priority date,filing/creation date,publication date,grant date,result link,representative figure link
0,CN-103890730-B,用于?感器??的?????用和服?的??和部署的?算平台,塔塔咨?服?有限公司,"普拉蒂普·米斯拉, 阿?潘·帕?, 巴拉穆拉利??·普?肖撒曼, 集拉布拉塔·包米克, 迪帕...",2011/9/19,2012/9/18,2017/7/11,2017/7/11,https://patents.google.com/patent/CN103890730B/zh,NaN
1,CN-103492389-B,"用作激?抑制?的?唑并[4,3-d]嘧啶",原真股份有限公司,"M·阿?姆施泰特?, M·托?曼, A·特雷姆?, N·特??",2011/4/21,2012/4/23,2016/9/14,2016/9/14,https://patents.google.com/patent/CN103492389B/zh,NaN


In [10]:
soup(data_cn["result link"][0])

{'patentNumber': 'CN103890730B',
 'inventorName': ['普拉蒂普·米斯拉',
  '阿尔潘·帕尔',
  '巴拉穆拉利达尔·普鲁肖撒曼',
  '集拉布拉塔·包米克',
  '迪帕克·斯瓦米',
  '文卡特拉马南·西瓦·苏布拉马尼亚恩',
  '阿维克·高斯',
  '阿尼鲁达·辛哈'],
 'current_assignee': ['Tata Consultancy Services Ltd'],
 'original_assignee': ['Tata Consultancy Services Ltd'],
 'appDate': '20120918',
 'pubDate': '20170711',
 'graDate': 'N/A',
 'classification': ['G06F8/00',
  'G06F11/3668',
  'G06F8/36',
  'G06F8/60',
  'H04L41/00',
  'H04L41/04',
  'H04L41/5096',
  'H04L43/00',
  'H04L67/10',
  'H04L67/12'],
 'brief': '本文公开了用于车辆遥测应用的智能开发、部署和管理的计算平台。此外，本发明提供了一种允许在基于云的平台上提供智能交通服务的方法和系统，该基于云的平台助于被配置为允许交通测量、交通整形、车辆监控和其他有关车辆的服务的车辆遥测应用的创建和部署。',
 'title': '用于传感器驱动的车辆遥测应用和服务的开发和部署的计算平台',
 'description': '',
 'claim': ['1.一种智能交通系统，包括；(a)基础设施平台，其包括多个硬件和软件组件、基础设施服务、应用程序接口API、软件开发工具包SDK，其中所述多个基础设施服务能够执行用于开发传感器驱动应用套件所需的算法、功能和调用；(b)多个传感器装置，其通过通信网络与所述基础设施平台间歇地连接，其中所述多个传感器能够将传感器数据反馈发送到所述基础设施服务云计算平台，其中所述传感器驱动应用套件被开发对应于所述传感器数据反馈；(c)车辆遥测应用开发装置，其与所述基础设施平台和所述多个传感器装置电子地耦合，其中所述车辆遥测应用开发装置使得应用开发者能够使用所述A

In [11]:
# MongoDB 連線
def insert_data(data):
    mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    conn_db_collection = db.patent_cn # collection 改成 CN
    
    conn_db_collection.insert_one(data) #insert data

In [ ]:
#爬蟲並新增資料
data_CN = pd.read_csv('../years/CN/CN_all.csv')
# urls = list(data.iloc[:, 0])
# del urls[0]

for i in tqdm(range(0,len(data_CN))): #len(data_CN)
    data_s = soup(data_CN["result link"][i])
    insert_data(data_s)

  3%|▎         | 1340/47313 [5:26:20<500:52:25, 39.22s/it]  

## US

In [9]:
data_US = pd.read_csv('../years/US/US_all.csv')

In [10]:
data_US.head()

,id,title,assignee,inventor/author,priority date,filing/creation date,publication date,grant date,result link,representative figure link
0,US-9990182-B2,Computer platform for development and deployme...,Tata Consultancy Services Limited,"Prateep Misra, Arpan Pal, Balamuralidhar Purus...",2011/9/19,2012/9/18,2018/6/5,2018/6/5,https://patents.google.com/patent/US9990182B2/en,https://patentimages.storage.googleapis.com/78...
1,US-2019027414-A1,Encapsulation barrier stack,"Agency For Science, Technology And Research, T...","Senthil Kumar Ramadas, Saravanan Shanmugavel",2011/10/24,2018/9/24,2019/1/24,NaN,https://patents.google.com/patent/US2019002741...,https://patentimages.storage.googleapis.com/05...
2,US-10216485-B2,Computer platform for development and deployme...,Tata Consultancy Services Limited,"Prateep Misra, Arpan Pal, Balamuralidhar Purus...",2011/9/19,2012/9/18,2019/2/26,2019/2/26,https://patents.google.com/patent/US10216485B2/en,https://patentimages.storage.googleapis.com/65...
3,US-2014310243-A1,Heart beacon cycle,Mr. Steven James McGee,"Steven James McGee, Tammy Lee McGee",2010/8/16,2012/8/13,2014/10/16,NaN,https://patents.google.com/patent/US2014031024...,https://patentimages.storage.googleapis.com/e2...
4,US-8965375-B2,Pathloss based access node wake-up control,Nokia Siemens Networks Oy,"Juergen Michel, Dirk Rose",2011/5/4,2011/5/4,2015/2/24,2015/2/24,https://patents.google.com/patent/US8965375B2/en,https://patentimages.storage.googleapis.com/5d...


In [33]:
data_US.iloc[7483:7484]

,id,title,assignee,inventor/author,priority date,filing/creation date,publication date,grant date,result link,representative figure link
7483,US-2021157312-A1,Intelligent vibration digital twin systems and...,"Strong Force Iot Portfolio 2016, Llc","Charles H. Cella, Gerald William Duffy, JR., J...",2016/5/9,2020/11/25,2021/5/27,NaN,https://patents.google.com/patent/US2021015731...,https://patentimages.storage.googleapis.com/59...


In [19]:
data_US[data_US.id == "US-2019174361-A1"]

,id,title,assignee,inventor/author,priority date,filing/creation date,publication date,grant date,result link,representative figure link
22301,US-2019174361-A1,"Methods, Wireless Device and Nodes for Managin...",Telefonaktiebolaget Lm Ericsson (Publ),"Jing Fu, Rafia Inam, Vlasios Tsiatsis, Aneta V...",2016/8/16,2016/8/16,2019/6/6,NaN,https://patents.google.com/patent/US2019017436...,https://patentimages.storage.googleapis.com/ab...


In [15]:
data_US.iloc[22300:22305]

,id,title,assignee,inventor/author,priority date,filing/creation date,publication date,grant date,result link,representative figure link
22300,US-10802554-B2,Cooling adapter for mobile device,"Nexar, Ltd.",Lev Yitzhak Lavy,2017/1/22,2019/6/24,2020/10/13,2020/10/13,https://patents.google.com/patent/US10802554B2/en,https://patentimages.storage.googleapis.com/4e...
22301,US-2019174361-A1,"Methods, Wireless Device and Nodes for Managin...",Telefonaktiebolaget Lm Ericsson (Publ),"Jing Fu, Rafia Inam, Vlasios Tsiatsis, Aneta V...",2016/8/16,2016/8/16,2019/6/6,NaN,https://patents.google.com/patent/US2019017436...,https://patentimages.storage.googleapis.com/ab...
22302,US-9986468-B2,Remove TCP deadlock caused by wireless device ...,"Futurewei Technologies, Inc.","Jing Yu, Yumin Tang",2016/3/18,2016/3/18,2018/5/29,2018/5/29,https://patents.google.com/patent/US9986468B2/en,https://patentimages.storage.googleapis.com/f3...
22303,US-2017272460-A1,Method and system of eliminating vulnerabiliti...,AO Kaspersky Lab,"Vyacheslav E. Rusakov, Marta Anna Janus",2016/3/18,2016/8/11,2017/9/21,NaN,https://patents.google.com/patent/US2017027246...,https://patentimages.storage.googleapis.com/c4...
22304,US-2019036004-A1,Strain sensitive piezoelectric system with opt...,Intel Corporation,"Feras Eid, Sasha N. OSTER, Shawna M. Liff, Joh...",2016/4/1,2016/4/1,2019/1/31,NaN,https://patents.google.com/patent/US2019003600...,https://patentimages.storage.googleapis.com/e8...


In [20]:
# MongoDB 連線
def insert_data(data):
    mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    conn_db_collection = db.patent_us # collection 改成 US
    
    conn_db_collection.insert_one(data) #insert data

In [21]:
#爬蟲並新增資料
data_US = pd.read_csv('../years/US/US_all.csv')
# urls = list(data.iloc[:, 0])
# del urls[0]

for i in tqdm(range(22303,len(data_US))):
    try:
        data_s = soup(data_US["result link"][i])
        insert_data(data_s)
    except:
        print("error===",i)
        pass

  3%|▎         | 784/28917 [1:57:48<23:31:01,  3.01s/it] 

error=== 23085
error=== 23086


 49%|████▉     | 14112/28917 [32:44:31<14:30:03,  3.53s/it]   

error=== 36415


 62%|██████▏   | 17993/28917 [36:50:18<15:10:57,  5.00s/it]   

...

 96%|█████████▌| 27715/28917 [45:51:19<00:31, 37.99it/s]

error=== 50013
error=== 50014
error=== 50015
error=== 50016
error=== 50017
error=== 50018
error=== 50019
error=== 50020


 96%|█████████▌| 27719/28917 [45:51:19<00:31, 38.06it/s]

error=== 50021


100%|██████████| 28917/28917 [47:07:42<00:00,  5.87s/it]  


In [ ]:
# US 3341 抓資料會錯誤
# 7483 檔案太大

In [ ]:
# Error 的資料
error=== 23085
error=== 23086
 49%|████▉     | 14112/28917 [32:44:31<14:30:03,  3.53s/it]   
error=== 36415
 62%|██████▏   | 17993/28917 [36:50:18<15:10:57,  5.00s/it]   
error=== 40296
 95%|█████████▌| 27587/28917 [45:51:15<2:47:44,  7.57s/it]   
error=== 49885
error=== 49886
error=== 49887
error=== 49888
error=== 49889
error=== 49890
error=== 49891
error=== 49892
 95%|█████████▌| 27595/28917 [45:51:15<1:21:59,  3.72s/it]
error=== 49893
error=== 49894
error=== 49895
error=== 49896
error=== 49897
error=== 49898
error=== 49899
error=== 49900
 95%|█████████▌| 27603/28917 [45:51:16<40:13,  1.84s/it]  
error=== 49901
error=== 49902
error=== 49903
error=== 49904
error=== 49905
error=== 49906
error=== 49907
error=== 49908
 95%|█████████▌| 27611/28917 [45:51:16<19:52,  1.09it/s]
error=== 49909
error=== 49910
error=== 49911
error=== 49912
error=== 49913
error=== 49914
error=== 49915
error=== 49916
 96%|█████████▌| 27619/28917 [45:51:16<09:58,  2.17it/s]
error=== 49917
error=== 49918
error=== 49919
error=== 49920
error=== 49921
error=== 49922
error=== 49923
error=== 49924
 96%|█████████▌| 27627/28917 [45:51:16<05:08,  4.18it/s]
error=== 49925
error=== 49926
error=== 49927
error=== 49928
error=== 49929
error=== 49930
error=== 49931
error=== 49932
 96%|█████████▌| 27635/28917 [45:51:16<02:47,  7.64it/s]
error=== 49933
error=== 49934
error=== 49935
error=== 49936
error=== 49937
error=== 49938
error=== 49939
error=== 49940
 96%|█████████▌| 27643/28917 [45:51:17<01:39, 12.84it/s]
error=== 49941
error=== 49942
error=== 49943
error=== 49944
error=== 49945
error=== 49946
error=== 49947
error=== 49948
 96%|█████████▌| 27651/28917 [45:51:17<01:05, 19.47it/s]
error=== 49949
error=== 49950
error=== 49951
error=== 49952
error=== 49953
error=== 49954
error=== 49955
error=== 49956
 96%|█████████▌| 27659/28917 [45:51:17<00:48, 25.89it/s]
error=== 49957
error=== 49958
error=== 49959
error=== 49960
error=== 49961
error=== 49962
error=== 49963
error=== 49964
 96%|█████████▌| 27667/28917 [45:51:17<00:40, 30.61it/s]
error=== 49965
error=== 49966
error=== 49967
error=== 49968
error=== 49969
error=== 49970
error=== 49971
error=== 49972
 96%|█████████▌| 27675/28917 [45:51:17<00:36, 33.94it/s]
error=== 49973
error=== 49974
error=== 49975
error=== 49976
error=== 49977
error=== 49978
error=== 49979
error=== 49980
 96%|█████████▌| 27683/28917 [45:51:18<00:34, 35.92it/s]
error=== 49981
error=== 49982
error=== 49983
error=== 49984
error=== 49985
error=== 49986
error=== 49987
error=== 49988
 96%|█████████▌| 27691/28917 [45:51:18<00:32, 37.15it/s]
error=== 49989
error=== 49990
error=== 49991
error=== 49992
error=== 49993
error=== 49994
error=== 49995
error=== 49996
 96%|█████████▌| 27699/28917 [45:51:18<00:32, 37.46it/s]
error=== 49997
error=== 49998
error=== 49999
error=== 50000
error=== 50001
error=== 50002
error=== 50003
error=== 50004
 96%|█████████▌| 27707/28917 [45:51:18<00:32, 37.51it/s]
error=== 50005
error=== 50006
error=== 50007
error=== 50008
error=== 50009
error=== 50010
error=== 50011
error=== 50012
 96%|█████████▌| 27715/28917 [45:51:19<00:31, 37.99it/s]
error=== 50013
error=== 50014
error=== 50015
error=== 50016
error=== 50017
error=== 50018
error=== 50019
error=== 50020
 96%|█████████▌| 27719/28917 [45:51:19<00:31, 38.06it/s]
error=== 50021

In [22]:
data_US["result link"][50021]

'https://patents.google.com/patent/US20200044934A1/en'

In [18]:
soup(data_US["result link"][22303])

{'patentNumber': 'US20170272460A1',
 'inventorName': ['Vyacheslav E. Rusakov', 'Marta Anna Janus'],
 'current_assignee': ['Kaspersky Lab AO'],
 'original_assignee': ['Kaspersky Lab AO'],
 'appDate': '20160811',
 'pubDate': '20170921',
 'graDate': '20171213',
 'classification': ['H04L63/1433',
  'G06F21/577',
  'H04L41/0809',
  'H04L67/02',
  'H04L67/06'],
 'brief': 'Disclosed are systems and methods for eliminating vulnerabilities of smart devices connected to a data network. An example method includes: identifying a router providing access to the data network, obtaining access to the network and transmitting a request through the data network to obtain access to a smart device on the network. Furthermore, the method includes accessing the smart device to obtain its settings, comparing the settings with known vulnerabilities, determining an action for repairing the a network vulnerability associated with the settings of the device, and transmitting instructions to the smart device to p